In [ ]:
print('INSTALANDO O XLSXWRITER PARA USAR COMO ENGINE AO SALVAR O ARQUIVO FINAL')

In [ ]:
pip install xlsxwriter

In [ ]:
print('IMPORTANDO AS BIBLIOTECAS NECESSÁRIAS...\n')

import xlsxwriter
import pandas as pd

In [ ]:
# ALGUMAS FUNÇÕES A SEREM UTILIZADAS

def so_str(df):
    
    '''
    Função para deixar somente letras na Razão Social
    
    INPUT: RECEBE O DF
    OUTPUT: DF COM A RAZÃO SOCIAL LIMPA
    '''
    
    new = []
    for i in df:
        
        try:
            new.append(''.join(char for char in i if char not in ['.','-','/','\\','!']))
        except (TypeError, ValueError):
            pass
        
    return(pd.Series(new))

In [ ]:
def limpa(df,lista):
    
    '''
    Função para retirar as colunas não utilizadas
    
    INPUT: RECEBE O DF E A LISTA DE COLUNAS ESSENCIAIS
    OUTPUT: RETORNA O DF SOMENTE COM AS COLUNAS ESSENCIAIS
    '''
    
    for i in df:
        if i not in lista:
            df.drop([i], axis = 1, inplace = True)
    
    df.reset_index(drop = True, inplace = True)

In [ ]:
def manipulação(fin):
    
    print('MANIPULANDO O RELATÓRIO FINANCEIRO...\n')

    # EXCLUI COLUNAS NÃO UTLIZADAS E PREENCHE VALORES VAZIOS

    lista = ['Razão Social','Valor Líquido', 'Data Pagamento','Observação','Nome Banco','Tipo Entrada']
    limpa(fin,lista)
    fin["Observação"].fillna('', inplace = True)
    fin.dropna(inplace = True)

    # TRANSFORMA OS VALORES DE STRING PARA FLOAT

    fun = lambda x: float(x.replace(".","").replace(",","."))
    fin["Valor Líquido"] = fin["Valor Líquido"].apply(fun)

    # RETIRA DA DATA O DIA DA SEMANA E SUBSTITUI A COLUNA NÃO FORMATADA

    fin["Data Pagamento"] = fin["Data Pagamento"].str.slice(stop = 10)
    fin.sort_values(by = ['Razão Social'], inplace = True)
    fin.reset_index(drop = True, inplace = True)

    # CRIA COLUNAS DE DÉBITO E CRÉDITO E PREENCHE COM A CONTA DE DUPLICATAS A COMPENSAR

    fin['DEBITO'] = ""
    fin['CREDITO'] = ''

    print('INSIRA A CONTA CONTÁBIL DE DUPLICATAS PAGAS A COMPENSAR:')
    x = int(input())
    print('\n')
    
    for i in range(len(fin['Nome Banco'])):
        if fin['Nome Banco'][i] not in ['CAIXA LETICIA','DANIELA','CAIXA GERENCIAL','COFRE']:
            fin['CREDITO'][i] = x
        else:
            fin['CREDITO'][i] = 5

In [ ]:
def preenche(fornec, fin):
    
    print('PREENCHENDO O RELATÓRIO FINANCEIRO COM AS CONTAS DOS FORNECEDORES... \n')
    
    # LIMPA OS FORNECEDORES

    fornec = fornec[:][11:-2]
    fornec['codigo'] = fornec['Empresa:']
    fornec['FORNECEDOR'] = fornec['Unnamed: 11']
    lista = ['codigo','FORNECEDOR']
    limpa(fornec, lista)
    
    # TRANSFORMA A COLUNA RAZÃO SOCIAL E FORNECEDOR EM UMA LISTA CADA E LIMPA OS CARACTERES ESPECIAIS

    razao1 = fin['Razão Social'].to_list()
    fin['Razão Social'] = so_str(razao1)

    razao = fornec['FORNECEDOR'].to_list()
    fornec['FORNECEDOR'] = so_str(razao)

    # ORGANIZA POR ORDEM ALFABÉTICA E TRANSFORMA EM DICIONÁRIO

    fornec.sort_values(by = ['FORNECEDOR'], inplace = True)
    fornec.reset_index(drop = True, inplace = True)
    fornec.dropna(inplace = True)
    dic = fornec.set_index('FORNECEDOR').T.to_dict('list')

    # PREENCHE O DF DE ACORDO COM A TABELA DE FORNECEDORES
    
    lista = ['BRADESCO SA','BANCO COOPERATIVO SICRED SA','CAIXA ECONOMICA FEDERAL SA',
             'COMPANHIA PAULSTA DE FORCA ELUZ CPFL','VR BENEF E SERV DE PROCESSAMENTO LTDA',
             'ARCAL SUPERMERCADO LTDA','ARCAL RESCISAO E PROCESSOS','ARLINDO CALSA FILHO',
             'BANCO DO BRASIL SA','MINISTERIO DA FAZENDA','SUPERMERCADO UNION',
             'COOPIDEAL SUPERMERCADOS EIRELI','REDE LOCAL']
    
    fin.reset_index(drop = True, inplace = True)

    for i in range(len(fin['Razão Social'])):
        if fin['Razão Social'][i] in dic and fin['Razão Social'][i] not in lista:
            fin['DEBITO'][i] = dic[fin['Razão Social'][i]][0]

    fin.sort_values(by=['Razão Social'], inplace = True)

    # LIMPA AS LINHAS QUE NÃO VÃO SER PREENCHIDAS POIS NÃO SÃO FORNECEDORES VÁLIDOS

    fin.reset_index(drop = True, inplace = True)

    for i in range(len(fin['Razão Social'])):

        if fin['Razão Social'][i] in lista:
            fin.drop(i, inplace = True)

In [ ]:
# FUNCTION TO SEPARATE THE DATAFRAME FOR STORE AND RETURNS A LIST WITH THE NAME OF EACH\n",

def prep_arq(df):

    loja_03_rede = df.loc[(df['Loja']) == 'CB1 PORTO FELIZ']
    deposito_rede = df.loc[(df['Loja']) == 'CB6 DEPOSITO']
    loja_05_rede = df.loc[(df['Loja']) == 'CB2 CERQUILHO']
    loja_08_rede = df.loc[(df['Loja']) == 'CB3 PIRA 08']
    loja_01_coop = df.loc[(df['Loja']) == 'CI LOJA 01']
    loja_04_coop = df.loc[(df['Loja']) == 'CB3 PIRA 04']
    loja_05_coop = df.loc[(df['Loja']) == 'CI LOJA 05']
    loja_07_coop = df.loc[(df['Loja']) == 'CI LOJA 07']
    loja_08_coop = df.loc[(df['Loja']) == 'CI LOJA 08']
    loja_09_coop = df.loc[(df['Loja']) == 'CB5 INDAIA']
    
    lojas = [deposito_rede, loja_08_rede, loja_05_rede, loja_03_rede, 
             loja_01_coop, loja_04_coop, loja_05_coop, loja_07_coop, loja_08_coop, loja_09_coop]

    nomes = ['deposito_rede', 'loja_08_rede', 'loja_05_rede', 'loja_03_rede', 
             'loja_01_coop', 'loja_04_coop', 'loja_05_coop', 'loja_07_coop', 'loja_08_coop', 'loja_09_coop']

    return(lojas, nomes)

In [ ]:
# SEPARATE EACH DATAFRAME SAVED WITH prep_arq() FOR BANK ACCOUNT AND SAVE IN .XLSX FORMAT\n",

def separa_contas(df, loja, fornec, x):

    with pd.ExcelWriter((f'{loja}.xlsx'), engine = 'xlsxwriter') as df_limpo:

        print("Prepares the df to fill the debit and credit columns\n")

        df['Valor Líquido'] = df['Valor Líquido'].apply(lambda x: float(x.replace('.','').replace(',','.')))

        df["Observação"].fillna('', inplace = True)
        df.dropna(inplace = True)

        df.sort_values(by = ['Razão Social'], inplace = True)
        df.reset_index(drop = True, inplace = True)

        # Creating columns

        df['Loja'] = df['Loja']
        df['importação'] = 'importação'
        df['Data'] = df['Data Pagto Contábil']
        df['Valor'] = df['Valor Líquido']
        df['Debito'] = '' # Will be fullfiled with provider code
        df['Credito'] = x
        df['Historico'] = df['Razão Social']
        df['Historico2'] = df['Tipo Entrada']
        df['Historico3'] = df['Observação']

        # TRANSFORMA A COLUNA RAZÃO SOCIAL E FORNECEDOR EM UMA LISTA CADA E LIMPA OS CARACTERES ESPECIAIS

        razao1 = df['Razão Social'].to_list()
        df['Razão Social'] = so_str(razao1)

        razao = fornec['FORNECEDOR'].to_list()
        fornec['FORNECEDOR'] = so_str(razao)

        # ORGANIZA POR ORDEM ALFABÉTICA E TRANSFORMA EM DICIONÁRIO

        fornec.sort_values(by = ['FORNECEDOR'], inplace = True)
        fornec.reset_index(drop = True, inplace = True)
        fornec.dropna(inplace = True)
        dic = fornec.set_index('FORNECEDOR').T.to_dict('list')

        print(f'PREENCHENDO O RELATÓRIO FINANCEIRO DA LOJA {loja} COM AS CONTAS DOS FORNECEDORES... \n')

        # PREENCHE O DF DE ACORDO COM A TABELA DE FORNECEDORES

        df.reset_index(drop = True, inplace = True)

        for i in range(len(df['Razão Social'])):
            if df['Razão Social'][i] in dic:
                df['Debito'][i] = dic[df['Razão Social'][i]][0]

        df.sort_values(by=['Razão Social'], inplace = True)

        # LIMPA AS LINHAS QUE NÃO VÃO SER PREENCHIDAS POIS NÃO SÃO FORNECEDORES VÁLIDOS

        df.reset_index(drop = True, inplace = True)

        lista = ['BANCO BRADESCO S/A.','BANCO COOPERATIVO SICRED SA','CAIXA ECONOMICA FEDERAL SA',
                'BANCO DO BRASIL SA','BANCO ITAU S/A','BANCO SAFRA S/A','BANCO SANTANDER S/A','BANCO TOPAZIO S.A.',
                'BANCO TRIANGULO S/A','CAIXA ECONOMICA FEDERAL-NOIVA DA COLINA']

        for i in range(len(df['Historico'])):

            if df['Historico'][i] in lista:
                df.drop(i, inplace = True)

        df.drop(['Data Pagto Contábil','Tipo Entrada','Observação',
                 'Valor Líquido','Razão Social', 'Loja'],axis = 1, inplace = True)

        # Separate the df by bank account

#             for i in range(len(bco)):
#                 s = bco[i]
#                 print(f'A {i + 1}ª conta é:{s}')
#                 print(f'O nome da aba na planilha ficou: {s}_Pgto_{loja}\n')
#                 bco[i] = df.loc[(df['Conta']) == s]
#                 bco[i].drop(['Conta'],axis = 1, inplace = True)
#                 bco[i].to_excel(df_limpo, sheet_name = (f'{s}Pgto{loja}'), index = False)

        print(f"The tab's name in the spredsheet is: Pgto_{loja}\n")
        df.to_excel(df_limpo, sheet_name = (f'Pgto_{loja}'), index = False)
        df_limpo.save()

In [ ]:
def resultado(fin):
    
    print('FORNECEDORES SEM PREENCHIMENTO: \n')

    # MOSTRA QUAIS FORNECEDORES FICARAM SEM PREENCHIMENTO

    forn_sem_conta = []

    fin.reset_index(inplace = True)

    for i in range(len(fin['DEBITO'])):
        if fin['DEBITO'][i] == '':
            forn_sem_conta.append(fin['Razão Social'][i])

    print(set(forn_sem_conta),'\n')

    # SOMA O TOTAL DE PAGAMENTOS REALIZADOS AOS FORNECEDORES

    soma = 0

    for i in range(len(fin['Tipo Entrada'])):

        if fin['Tipo Entrada'][i] in ['COMERCIALIZACAO E REVENDA', 'COMPRA DE MERCADORIAS', 
                                      'COMPRA DE MERCADORIAS P/ PRODUCAO INTERNA']:
            soma += fin['Valor Líquido'][i]

    print('Total de pagamentos realizados no mês: R$ %.2f'%(soma),'\n')

    # MOSTRA QUANTAS LINHAS FICARAM SEM PREENCHIMENTO

    soma = 0
    for i in range(len(fin['DEBITO'])):
        if fin['DEBITO'][i] == '':
            soma +=1

    print(f'Total de linhas sem preenchimento: {soma}\n\nTotal de fornecedores sem preenchimento: {len(set(forn_sem_conta))}\n')


# Fazer um pacote e criar funções dos processos da transformação

# Tem que ter um arquivo __init__.py
# Fazer o pacote e subir no Github todos os arquivos

# Tentar capturar os erros


In [ ]:
def exporta(fin):
    
    # EXPORTAR PARA NOVO ARQUIVO

    # fin.drop('index', axis = 1, inplace = True)

    fin_limpo = pd.ExcelWriter('arq_limpo.xlsx', engine = 'xlsxwriter')
    fin.to_excel(fin_limpo, index = False,  float_format="%.2f")
    fin_limpo.save()

In [ ]:
# RODANDO TODO O PROCESSO DE MANIPULAÇÃO E PREENCHIMENTO DOS ARQUIVOS E EXPORTANDO NO FORMATO .XLSX

continuar = 'sim'

while True:
    
    if continuar.lower() == 'sim':
        
        print('Preencha o nome do arquivo com a lista de fornecedores:')
        print('Se for do grupo coopideal/rede local, especificar com _coop ou _rede no final.')
        balancete = input()
        print('\n')
        print('Preencha o nome do arquivo do relatório financeiro:')
        relatorio = input()
        print('\n')
        print('IMPORTANDO AS TABELAS...\n')
        
        fornec = pd.read_csv(f'{balancete}.csv', encoding = 'iso-8859-1', error_bad_lines = False, sep = ';')
        fin = pd.read_csv(f'{relatorio}.csv', sep = ';', encoding = 'iso-8859-1', error_bad_lines = False)
        
        if '_coop' in balancete or '_rede' in balancete:

            print('MANIPULANDO O RELATÓRIO FINANCEIRO...\n')
            df, lojas = prep_arq(fin)

            print('INSIRA A CONTA CONTÁBIL DE DUPLICATAS PAGAS A COMPENSAR:')
            conta_duplics = int(input())

            for i in range(len(df)):

                if lojas[i].endsWith('_rede'):
                    fornecedor = pd.read_csv(f'{rede}.csv', encoding = 'iso-8859-1', error_bad_lines = False, sep = ';')
                    
                else:
                    fornecedor = pd.read_csv(f'{coop}.csv', encoding = 'iso-8859-1', error_bad_lines = False, sep = ';')

                separa_contas(df[i], lojas[i], fornecedor, conta_duplics)

            print('Arquivos salvos!')
            print('Mais arquivos? Sim/Não')
            continuar = input()
        
        else:
            
            manipulação(fin)
            preenche(fornec, fin)
            exporta(fin)
            resultados(fin)

            print('Arquivo(s) salvo(s)!')
            print('Mais arquivos? Sim/Não')
            continuar = input()
    
    elif continuar.lower() == 'não':
        break
    
    else:
        print('Comando incorreto! Por favor preencha Sim ou Não.\n')
        continuar = input()